# Mean-Field AOA

In [1]:
import numpy as np
import h5py

In [2]:
import sys
sys.path.append('../src')
from mf_AOA import *

## Sherrington-Kirkpatrick

In [3]:
# schedule
p = 1000
τ = 0.5

γ = τ * (np.arange(1, p + 1) - 1/2) / p
β = τ * (1 - np.arange(1, p + 1) / p)
β[p-1] = τ / (4 * p)

In [4]:
# path to folder with the h5 files
PATH_DB = "/home/lappet/Dropbox/MidnightRambler/Mean_Field_AOA/Figures/Data/SK_model/Averaged_MF_energy/"
PATH = "/home/lappet/Archives/projects/QuantumComputing/QAA_and_MFQAOA/data/mean-field-qaoa/results/sherrington-kirkpatrick/new/"

In [5]:
seed = 137
np.random.seed(seed)

### Raw data

In [6]:
# for num in [10 + k * 5 for k in range(23)]:
for num in [9]:
    N = num
    num_instances = 10000

    file = "SK_model_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + ".h5"
    print(file)

    h5_file = h5py.File(PATH + file, "w")

    # Sherrington-Kirkpatrick model
    from tqdm import tqdm
    for i in tqdm(range(num_instances)):
        J = np.random.normal(0, 1, size=(N, N)) / np.sqrt(N)
        J = np.triu(J, k=1)
        J = J + J.transpose()

        S = np.array([[1., 0., 0.] for _ in range(N - 1)]) # fix final spin (i.e. leave it out)
        S = evolve(S, J, β, γ)

        h5_file.create_dataset("set_" + str(i + 1) + "/J", data=J)
        h5_file.create_dataset("set_" + str(i + 1) + "/S_z", data=S[:, 2])

    h5_file.close()

  0%|          | 2/10000 [00:00<10:36, 15.71it/s]

SK_model_p_1000_N_9_num_inst_10000_seed_137.h5


100%|██████████| 10000/10000 [07:33<00:00, 22.04it/s]


### Histogram statistics

In [7]:
seed = 137
np.random.seed(seed)

N = 9
num_instances = 10000

file = "SK_model_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + ".h5"
print(file)    
data_file = h5py.File(PATH + file, "r")

E_stars = []
for i in range(num_instances):
    J = data_file["set_" + str(i + 1) + "/J"][:]
    S_z = data_file["set_" + str(i + 1) + "/S_z"][:]

    E_star = expectation(solution(S_z), J)
    E_stars.append(E_star)

file = "SK_model_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + "_all_E_stars.h5"    
out_file = h5py.File(PATH_DB + file, "w")
out_file.create_dataset("all_E_stars", data=E_stars)
out_file.close()

SK_model_p_1000_N_9_num_inst_10000_seed_137.h5


### Moments

In [ ]:
E_0 = -0.763

for num in [10 + k * 5 for k in range(23)]:
    N = num
    num_instances = 10000

    file = "SK_model_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + ".h5"
    print(file)    
    data_file = h5py.File(PATH + file, "r")

    E_star = 0
    E_star_squared = 0
    for i in range(num_instances):
        J = data_file["set_" + str(i + 1) + "/J"][:]
        S_z = data_file["set_" + str(i + 1) + "/S_z"][:]

        E_star += expectation(solution(S_z), J)
        E_star_squared += expectation(solution(S_z), J)**2
        
    out_file = h5py.File(PATH_DB + file, "w")
    out_file.create_dataset("E_star", data=[E_star / num_instances])
    out_file.create_dataset("E_star_squared", data=[E_star_squared / num_instances])
    out_file.close()

## Number Partitioning

In [ ]:
# schedule
p = 10000
τ = 0.25

γ = τ * (np.arange(1, p + 1) - 1/2) / p
β = τ * (1 - np.arange(1, p + 1) / p)
β[p-1] = τ / (4 * p)

### Raw data

In [ ]:
# PATH = "../data/number-partitioning/instances/batch_06/"

# X = np.linspace(4, 20, 9, dtype=int)
# ENSEMBLE_SIZE = 100

# for n in X:
#     for i in range(ENSEMBLE_SIZE):
        
#         filename = "number_partitioning_num_{0:02d}".format(n) + "_replica_{0:03d}".format(i + 1)
#         print(i, ", ", filename, "\n")
        
#         a = np.load(PATH + filename + ".npy")
#         J = -2 * np.outer(a.T, a)
#         np.fill_diagonal(J, 0.)    
        
#         # freeze final spin to +1
#         h_z = J[-1, :-1]
#         J = J[:-1, :-1]
#         print(J, "\n")

In [4]:
PATH = "/home/lappet/Archives/projects/QuantumComputing/QAA_and_MFQAOA/data/mean-field-qaoa/results/number-partitioning/batch_06/new/"
PATH_DB = "/home/lappet/Dropbox/MidnightRambler/Mean_Field_AOA/Figures/Data/NP_problem/new/"

In [5]:
X = np.linspace(4, 20, 9, dtype=int)

for N in X:
    seed = 137
    np.random.seed(seed)
    num_instances = 10000

    file = "number-partitioning_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + ".h5"
    print(file)

    h5_file = h5py.File(PATH + file, "w")

    # Number partitioning
    from tqdm import tqdm
    for i in tqdm(range(num_instances)):
        a = np.random.uniform(0, 1, size=N) 
        a = np.sort(a)
        J = -2 * np.outer(a.T, a)
        np.fill_diagonal(J, 0.)    
        
        S = np.array([[1., 0., 0.] for _ in range(N - 1)]) # fix final spin (i.e. leave it out)
        S = evolve(S, J, β, γ)

        h5_file.create_dataset("set_" + str(i + 1) + "/a", data=a)
        h5_file.create_dataset("set_" + str(i + 1) + "/S_z", data=S[:, 2])

    h5_file.close()        

  0%|          | 2/10000 [00:00<08:25, 19.77it/s]

number-partitioning_p_1000_N_4_num_inst_10000_seed_137.h5


  0%|          | 3/10000 [00:00<06:40, 24.94it/s]

number-partitioning_p_1000_N_6_num_inst_10000_seed_137.h5


  0%|          | 3/10000 [00:00<06:45, 24.65it/s]

...

  0%|          | 3/10000 [00:00<06:37, 25.14it/s]

number-partitioning_p_1000_N_18_num_inst_10000_seed_137.h5


  0%|          | 3/10000 [00:00<07:05, 23.47it/s]

number-partitioning_p_1000_N_20_num_inst_10000_seed_137.h5


100%|██████████| 10000/10000 [06:32<00:00, 25.48it/s]


### Histogram statistics

In [123]:
seed = 137
np.random.seed(seed)

N = 10
num_instances = 10000

file = "number-partitioning_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + ".h5"
print(file)    
data_file = h5py.File(PATH + file, "r")

E_stars = []
for i in range(num_instances):
    a = data_file["set_" + str(i + 1) + "/a"][:]
    S_z = data_file["set_" + str(i + 1) + "/S_z"][:]
    
    J = -2 * np.outer(a.T, a)
    np.fill_diagonal(J, 0.)   
    
    import copy
    E_and_string = (np.sum(a**2) + expectation(solution(S_z), J), S_z)
    
#     # one spin flip
#     for j in range(S_z.shape[0]):
#         new_S_z = copy.deepcopy(S_z)
#         new_S_z[j] *= -1
#         if E_and_string[0] > np.sum(a**2) + expectation(solution(new_S_z), J):
#             E_and_string = (np.sum(a**2) + expectation(solution(new_S_z), J), new_S_z)

    # two spin flips
    for j in range(S_z.shape[0]):
        for k in range(j + 1, S_z.shape[0]):
            new_S_z = copy.deepcopy(S_z)
            new_S_z[j] *= -1
            new_S_z[k] *= -1
            if E_and_string[0] > np.sum(a**2) + expectation(solution(new_S_z), J):
                E_and_string = (np.sum(a**2) + expectation(solution(new_S_z), J), new_S_z)
    
    E_stars.append(np.sqrt(E_and_string[0]))

file = "number-partitioning_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + "_all_E_stars.h5"    
out_file = h5py.File(PATH_DB + file, "w")
out_file.create_dataset("all_E_stars", data=E_stars)
out_file.close()

number-partitioning_p_1000_N_10_num_inst_10000_seed_137.h5


### Moments

In [ ]:
E_0 = -0.763

for N in X:
    num_instances = 10000

    file = "number-partitioning_" + "p_" + str(p) + "_N_" + str(N) + "_num_inst_" + str(num_instances) + "_seed_" + str(seed) + ".h5"
    print(file)    
    data_file = h5py.File(PATH + file, "r")

    E_star_sqrt = 0
    E_star_squared = 0
    for i in range(num_instances):
        a = data_file["set_" + str(i + 1) + "/a"][:]
        S_z = data_file["set_" + str(i + 1) + "/S_z"][:]

        J = -2 * np.outer(a.T, a)
        np.fill_diagonal(J, 0.)

        E_star_sqrt += np.sqrt(np.sum(a**2) + expectation(solution(S_z), J))
#         E_star_squared += expectation(solution(S_z), J)**2
        
    out_file = h5py.File(PATH_DB + file, "w")
    out_file.create_dataset("E_star_sqrt", data=[E_star_sqrt / num_instances])
#     out_file.create_dataset("E_star_squared", data=[E_star_squared / num_instances])
    out_file.close()